In [1]:
import sys
sys.path.append('../../build')
import libry as ry

import cv2 as cv
from ipywidgets import interact
import numpy as np
import time


In [2]:
C = ry.Config()
C.addFile('../../../scenarios/pandasTable.g')
V = ry.ConfigurationViewer()
cameraFrame = C.frame("camera")

q0 = C.getJointState()
R_gripper = C.frame("R_gripper")
R_gripper.setContact(1)
L_gripper = C.frame("L_gripper")
L_gripper.setContact(1)

V.setConfiguration(C)

In [3]:
#set  flat object on top of a table on the table edge

obj = C.addFrame("Objects")
obj.setColor([1, 0, 0])
obj.setShape(ry.ST.ssBox, [0.2, 0.2, 0.1, 0.01])
obj.setPosition([.7, 0, .7])
obj.setMass(1)
obj.setContact(1)

#@interact(x=(-1, 1, 0.1), y = (-1, 1, 0.1), z = (-1, 2, 0.1), a = (0, 1, 0.1))
#def set_position(x, y, z):
#    obj.setPosition([x, y, z])
#    V.setConfiguration(C)

X0 = C.getFrameState()

In [4]:
S = C.simulation(ry.SimulatorEngine.physx, True)
S.addSensor("camera")

In [5]:
#the focal length
f = 0.895
f = f * 360.
#the relative pose of the camera
# pcl.setRelativePose('d(-90 0 0 1) t(-.08 .205 .115) d(26 1 0 0) d(-1 0 1 0) d(6 0 0 1) ')
fxfypxpy = [f, f, 320., 180.]

In [6]:

points = []
tau = .01

for t in range(300):
    time.sleep(0.03)
    #grab sensor readings from the simulation
    q = S.get_q()
    if t%10 == 0:

        [rgb, depth] = S.getImageAndDepth()  #we don't need images with 100Hz, rendering is slow
        points = S.depthData2pointCloud(depth, fxfypxpy)
        cameraFrame.setPointCloud(points, rgb)
        V.recopyMeshes(C)
        V.setConfiguration(C)

        #use hsv - simpler for color finding
        hsv = cv.cvtColor(rgb, cv.COLOR_BGR2HSV)

        lower_red = np.array([115, 100, 100])
        upper_red = np.array([130, 255, 255])

        mask = cv.inRange(hsv, lower_red, upper_red)

        edges = cv.Canny(mask, 20, 60)
        #find contours adn draw in rgb image
        contours, hierarchy = cv.findContours(mask, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)
        cv.drawContours(rgb, contours, -1, (0, 255, 0), 2)

        for cnt in contours:
            approx = cv.approxPolyDP(cnt, 0.05 * cv.arcLength(cnt,True),True)
            hull = cv.convexHull(approx)
            #print(len(approx))
            M = cv.moments(approx)
            if M["m00"] != 0:
                cX = int(M["m10"] / M["m00"])
                cY = int(M["m01"] / M["m00"])
                # draw the contour and center of the shape on the image
                cv.circle(rgb, (cX, cY), 7, (255, 255, 255), -1)
                cv.putText(rgb, "center", (cX - 20, cY - 20),
                           cv.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)

                cv.drawContours(rgb, hull, -1, (255, 0, 255), 3)
                
                # corner points of the object
                point1 = points[hull[0][0][1]][hull[0][0][0]]
                point2 = points[hull[1][0][1]][hull[1][0][0]]
                point3 = points[hull[2][0][1]][hull[2][0][0]]
                point4 = points[hull[3][0][1]][hull[3][0][0]]

                # calculate distance between the points to get edges length of the centroids
                #print("point1 " + str(point1))
                #print("point2 " + str(point2))
                dist1 = np.linalg.norm(point1-point2)
                dist2 = np.linalg.norm(point1-point4)
                vec1 = point1 - point2
                vec2 = point1 - point4
                #print(vec1)


        if len(rgb)>0: cv.imshow('OPENCV - rgb', rgb)
        if len(mask)>0: cv.imshow('OPENCV - mask', mask)



        if cv.waitKey(1) & 0xFF == ord('q'):
            break


    S.step([], tau, ry.ControlMode.none)
